In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("anime.csv")

Para poder separar los generos de cada serie, necesito primero pasar la cadena a una lista. Entremedio, proceso los generos para eliminar los espacios a los costados y las mayusculas.

In [3]:
df.Genres.map(lambda x: [ y.strip().lower() for y in x.split(",")])

0        [action, adventure, comedy, drama, sci-fi, space]
1                  [action, drama, mystery, sci-fi, space]
2        [action, sci-fi, adventure, comedy, drama, sho...
3        [action, mystery, police, supernatural, drama,...
4              [adventure, fantasy, shounen, supernatural]
                               ...                        
17557                   [adventure, mystery, supernatural]
17558                       [comedy, horror, supernatural]
17559    [mystery, dementia, horror, psychological, sup...
17560                   [adventure, slice of life, comedy]
17561                                    [action, fantasy]
Name: Genres, Length: 17562, dtype: object

In [4]:
df["generos"] = df.Genres.map(lambda x: [ y.strip().lower() for y in x.split(",")])

Con los datos en una lista, ahora puedo aplicar explode para que cada valor se vuelva una fila y se una a los datos anteriores

In [5]:
expandido = df.explode("generos")

Notar que Cowboy Bebop esta asociado a 6 generos distintos, por ende ahora en nuestro nuevo DataFrame, va a aparecer 6 veces, cada una asociada a uno de los generos que le corresponde en la columna "genero"

In [6]:
expandido.set_index("Name").loc["Cowboy Bebop"]

,MAL_ID,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,generos
Name,,,,,,,,,,,,,,,,,,,,,
Cowboy Bebop,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,action
Cowboy Bebop,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,adventure
Cowboy Bebop,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,comedy
Cowboy Bebop,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,drama
Cowboy Bebop,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,sci-fi
Cowboy Bebop,1,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,...,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,space


Nota: si no usaba el set index, no podia despues usar loc por serie

Antes de seguir, normalizo los nombres pasandolos todos a minusculas

In [7]:
expandido["name"] = expandido.Name.map(lambda x:x.lower())

Ahora lo que quiero es tener en una representacion matricial, para cada serie, si pertenece a un genero determinado. En este caso para eso vamos a utilizar pivot porque cada serie solo deberia aparecer una unica vez asociada a un genero. Elegimos la columna de episodios como valores por ahora unicamente porque ya sabemos que va a pasar a continuacion

In [8]:
expandido.pivot("name","generos","Episodes")

ValueError: ignored

Pivot va a fallar porque dice tener valores repetidos. Esto significa que existe al menos un par nombre,genero que aparece mas de una vez. Por precondicion no deberia ocurrir, porque las series no deberian repetirse y los generos que extrajimos son unicos. Este tipo de cosas nos hablan de que hay errores en el dataset.

Notar que si hubieramos utilizado pivot table, el error no hubiera ocurrido porque por defecto va a intentar promediar los valores y al ser la cantidad de episodios no iba a fallar. Al margen de las cuestiones de optimizacion, si estamos seguros que los datos tienen que cumplir cierta precondicion, es mejor utilizar cosas especificas.

Probamos entonces utilizando pivot table, pero utilizando como funcion de agregacion count para contar los elementos y asi encontrar los pares series-genero que se repiten

In [11]:
df_count = expandido.pivot_table(index="name",columns="generos",values="Score",aggfunc="count")
df_count

generos,action,adventure,cars,comedy,dementia,demons,drama,ecchi,fantasy,game,...,slice of life,space,sports,super power,supernatural,thriller,unknown,vampire,yaoi,yuri
name,,,,,,,,,,,,,,,,,,,,,
"""0""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""aesop"" no ohanashi yori: ushi to kaeru, yokubatta inu",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""bungaku shoujo"" kyou no oyatsu: hatsukoi",NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""bungaku shoujo"" memoire",NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""bungaku shoujo"" movie",NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zutto mae kara suki deshita.: kokuhaku jikkou iinkai - kinyoubi no ohayou,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zutto suki datta,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
üks uks,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_count.max()

generos
action           2.0
adventure        1.0
cars             1.0
comedy           1.0
dementia         1.0
demons           2.0
drama            1.0
ecchi            1.0
fantasy          3.0
game             1.0
harem            1.0
hentai           1.0
historical       1.0
horror           1.0
josei            1.0
kids             2.0
magic            3.0
martial arts     1.0
mecha            1.0
military         1.0
music            1.0
mystery          1.0
parody           1.0
police           1.0
psychological    1.0
romance          1.0
samurai          1.0
school           3.0
sci-fi           1.0
seinen           1.0
shoujo           1.0
shoujo ai        1.0
shounen          1.0
shounen ai       1.0
slice of life    1.0
space            1.0
sports           1.0
super power      1.0
supernatural     1.0
thriller         1.0
unknown          1.0
vampire          1.0
yaoi             1.0
yuri             1.0
dtype: float64

Vemos que por ejemplo 'magic' tiene al menos una serie asociada 3 veces, si miramos

In [14]:
df_count.loc[df_count["magic"] > 1]

generos,action,adventure,cars,comedy,dementia,demons,drama,ecchi,fantasy,game,...,slice of life,space,sports,super power,supernatural,thriller,unknown,vampire,yaoi,yuri
name,,,,,,,,,,,,,,,,,,,,,
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Si revisamos en el expandido, vemos que la serie aparece multiples veces, pero algunos de esos datos son inconsistentes. Esto nos puede llevar a pensar que el dataset esta mal, ya que no podemos con la informacion que tenemos saber el por que de los datos duplicados

In [15]:
expandido.set_index("name").loc["maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e"]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,generos
name,,,,,,,,,,,,,,,,,,,,,
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,7.34,"Action, Demons, Magic, Fantasy, School",The Misfit of Demon King Academy,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,13,"Jul 4, 2020 to Sep 26, 2020",Summer 2020,...,23990.0,50133.0,58622.0,27479.0,12119.0,5087.0,2364.0,1209.0,835.0,action
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,7.34,"Action, Demons, Magic, Fantasy, School",The Misfit of Demon King Academy,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,13,"Jul 4, 2020 to Sep 26, 2020",Summer 2020,...,23990.0,50133.0,58622.0,27479.0,12119.0,5087.0,2364.0,1209.0,835.0,demons
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,7.34,"Action, Demons, Magic, Fantasy, School",The Misfit of Demon King Academy,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,13,"Jul 4, 2020 to Sep 26, 2020",Summer 2020,...,23990.0,50133.0,58622.0,27479.0,12119.0,5087.0,2364.0,1209.0,835.0,magic
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,7.34,"Action, Demons, Magic, Fantasy, School",The Misfit of Demon King Academy,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,13,"Jul 4, 2020 to Sep 26, 2020",Summer 2020,...,23990.0,50133.0,58622.0,27479.0,12119.0,5087.0,2364.0,1209.0,835.0,fantasy
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",40496,Maou Gakuin no Futekigousha: Shijou Saikyou no...,7.34,"Action, Demons, Magic, Fantasy, School",The Misfit of Demon King Academy,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,13,"Jul 4, 2020 to Sep 26, 2020",Summer 2020,...,23990.0,50133.0,58622.0,27479.0,12119.0,5087.0,2364.0,1209.0,835.0,school
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",48417,Maou Gakuin no Futekigousha: Shijou Saikyou no...,Unknown,"Magic, Fantasy, School",Unknown,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,magic
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",48417,Maou Gakuin no Futekigousha: Shijou Saikyou no...,Unknown,"Magic, Fantasy, School",Unknown,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,fantasy
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",48417,Maou Gakuin no Futekigousha: Shijou Saikyou no...,Unknown,"Magic, Fantasy, School",Unknown,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,school
"maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e",48418,Maou Gakuin no Futekigousha: Shijou Saikyou no...,Unknown,"Action, Demons, Magic, Fantasy, School",Unknown,魔王学院の不適合者 ～史上最強の魔王の始祖、転生して子孫たちの学校へ通う～,TV,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,action


Con esto, sabemos que tenemos que buscar las series que aparecen mas de una vez, pero para ello deberiamos agrupar y despues aplicar un count.

In [17]:
df.groupby("Name").count()

,MAL_ID,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,...,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,generos
Name,,,,,,,,,,,,,,,,,,,,,
"""0""",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"""Bungaku Shoujo"" Memoire",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"""Bungaku Shoujo"" Movie",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Rou,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
xxxHOLiC Shunmuki,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Üks Uks,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


No tiene mucho sentido hacer un segundo df con los counts para unirlo a los datos originales, entonces es mejor utilizar transform que ya nos genera los datos para incluir en el DataFrame utilizando la informacion que almacena el objeto Group

In [19]:
df["apariciones"] = df.groupby("Name").transform("count")["Score"]

Si eliminamos todos los invalidos, ahora podemos volver a hacer lo que hicimos al principio para llegar a aplicar pivot

In [20]:
df["invalido"] = df.apariciones > 1
df = df[~df["invalido"]]

In [21]:
df["generos"] = df.Genres.map(lambda x: [ y.strip().lower() for y in x.split(",")])
expandido = df.explode("generos")
expandido["name"] = expandido.Name.map(lambda x:x.lower())

Ahora si, antes de hacer el pivot vamos a pensar mejor que es lo que queremos obtener. Estamos buscando llegar a un DataFrame que para cada serie, tenga para cada uno de los generos si pertenece a ese o no. Como lo que vamos a querer que aparezca en mi nuevo DataFrame son valores booleanos, y sabiendo que en _expandido_ estan todos los pares serie-genero que son validos; puedo agregar una columna solo con True para que al generar el pivot me queden los valores True en los pares validos.

De la misma forma, sabemos que si en _expandido_ no aparece un determinado par serie-genero, el mismo es implicitamente False, entonces al pivot le puedo hacer un fillna por ese valor

In [29]:
expandido["t"] = True

serie_genero = expandido.pivot("name","generos","t")


serie_genero = serie_genero.fillna(False)

Con este formato puedo obtener facilmente para cualquier serie si pertenece a un determinado genero. Y como puedo hacer eso, puedo utilizarlo para filtrar los otros DataFrame

In [30]:
serie_genero.loc["ēldlive","game"]

False

In [32]:
def es_tipo(serie,tipo):
    return serie_genero.loc[serie,tipo]

In [33]:
expandido[ expandido.name.map( lambda x: es_tipo(x,"game"))]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-5,Score-4,Score-3,Score-2,Score-1,generos,apariciones,invalido,name,t
29,48,.hack//Sign,6.98,"Game, Sci-Fi, Adventure, Mystery, Magic, Fantasy",.hack//Sign,.hack//SIGN,TV,26,"Apr 4, 2002 to Sep 26, 2002",Spring 2002,...,7610.0,3688.0,1502.0,825.0,619.0,game,1,False,.hack//sign,True
29,48,.hack//Sign,6.98,"Game, Sci-Fi, Adventure, Mystery, Magic, Fantasy",.hack//Sign,.hack//SIGN,TV,26,"Apr 4, 2002 to Sep 26, 2002",Spring 2002,...,7610.0,3688.0,1502.0,825.0,619.0,sci-fi,1,False,.hack//sign,True
29,48,.hack//Sign,6.98,"Game, Sci-Fi, Adventure, Mystery, Magic, Fantasy",.hack//Sign,.hack//SIGN,TV,26,"Apr 4, 2002 to Sep 26, 2002",Spring 2002,...,7610.0,3688.0,1502.0,825.0,619.0,adventure,1,False,.hack//sign,True
29,48,.hack//Sign,6.98,"Game, Sci-Fi, Adventure, Mystery, Magic, Fantasy",.hack//Sign,.hack//SIGN,TV,26,"Apr 4, 2002 to Sep 26, 2002",Spring 2002,...,7610.0,3688.0,1502.0,825.0,619.0,mystery,1,False,.hack//sign,True
29,48,.hack//Sign,6.98,"Game, Sci-Fi, Adventure, Mystery, Magic, Fantasy",.hack//Sign,.hack//SIGN,TV,26,"Apr 4, 2002 to Sep 26, 2002",Spring 2002,...,7610.0,3688.0,1502.0,825.0,619.0,magic,1,False,.hack//sign,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17477,46488,Tai-Ari deshita.: Ojou-sama wa Kakutou Game na...,Unknown,"Game, Comedy, School, Seinen, Shoujo Ai",Unknown,対ありでした。 ～お嬢さまは格闘ゲームなんてしない～,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,seinen,1,False,tai-ari deshita.: ojou-sama wa kakutou game na...,True
17477,46488,Tai-Ari deshita.: Ojou-sama wa Kakutou Game na...,Unknown,"Game, Comedy, School, Seinen, Shoujo Ai",Unknown,対ありでした。 ～お嬢さまは格闘ゲームなんてしない～,Unknown,Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,shoujo ai,1,False,tai-ari deshita.: ojou-sama wa kakutou game na...,True
17535,48391,Mazica Party,Unknown,"Game, Magic, Fantasy",Unknown,マジカパーティ,TV,Unknown,"Apr 4, 2021 to ?",Spring 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,game,1,False,mazica party,True
17535,48391,Mazica Party,Unknown,"Game, Magic, Fantasy",Unknown,マジカパーティ,TV,Unknown,"Apr 4, 2021 to ?",Spring 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,magic,1,False,mazica party,True


Quiza para un unico valor no tiene mucho sentido, pero este formato me permite hacer cosas un poco mas avanzadas como generarme una funcion que me devuelva si aparece alguno de los generos de una lista arbitraria

In [50]:
def tiene_generos(fila,generos):  
    for genero in generos:
        if serie_genero.loc[fila["name"],genero]:
            return True
    return False

df[df.apply(lambda x:tiene_generos(x,["drama","game"]),axis=1)]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,generos,apariciones,invalido,name
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,20688.0,8904.0,3184.0,1357.0,741.0,1580.0,"[action, adventure, comedy, drama, sci-fi, space]",1,False,cowboy bebop
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,5805.0,1877.0,577.0,221.0,109.0,379.0,"[action, drama, mystery, sci-fi, space]",1,False,cowboy bebop: tengoku no tobira
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,15376.0,5838.0,1965.0,664.0,316.0,533.0,"[action, sci-fi, adventure, comedy, drama, sho...",1,False,trigun
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,5709.0,2920.0,1083.0,353.0,164.0,131.0,"[action, mystery, police, supernatural, drama,...",1,False,witch hunter robin
6,16,Hachimitsu to Clover,8.06,"Comedy, Drama, Josei, Romance, Slice of Life",Honey and Clover,ハチミツとクローバー,TV,24,"Apr 15, 2005 to Sep 27, 2005",Spring 2005,...,5574.0,3148.0,1339.0,484.0,278.0,321.0,"[comedy, drama, josei, romance, slice of life]",1,False,hachimitsu to clover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17501,47257,Shinigami Bocchan to Kuro Maid,Unknown,"Comedy, Drama, Romance",The Duke of Death and His Maid,死神坊ちゃんと黒メイド,TV,Unknown,2021 to ?,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"[comedy, drama, romance]",1,False,shinigami bocchan to kuro maid
17518,47777,Toku: Touken Ranbu - Hanamaru - Setsugetsuka,Unknown,"Action, Slice of Life, Comedy, Historical, Dra...",Unknown,特『刀剣乱舞-花丸-』～雪月華～,Movie,3,2022,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"[action, slice of life, comedy, historical, dr...",1,False,toku: touken ranbu - hanamaru - setsugetsuka
17526,48177,Ichiban Chikakute Tooi Hoshi,Unknown,"Music, Drama",Unknown,一番近くて遠い星,ONA,1,"Feb 18, 2021",Unknown,...,11.0,10.0,1.0,1.0,Unknown,Unknown,"[music, drama]",1,False,ichiban chikakute tooi hoshi
17535,48391,Mazica Party,Unknown,"Game, Magic, Fantasy",Unknown,マジカパーティ,TV,Unknown,"Apr 4, 2021 to ?",Spring 2021,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,"[game, magic, fantasy]",1,False,mazica party


Puedo reescribir la funcion para en vez de utilizar apply, operar directamente entre columnas. Es claramente mas complicado de pensar en este formato, pero es posible

In [51]:
def tiene_generos_v2(generos):
    result = serie_genero[generos[0]]

    for x in generos[1:]:
        result = result | serie_genero[x] 
    
    return result

def filtrar(frame):
    result = tiene_generos_v2(["drama","game"]).to_frame()

    result.sort_values("name",inplace=True)

    df["name"] = df.Name.map(lambda x: x.lower())
    tmp = df.sort_values("name")
    return tmp[ list(result[0]) ]

filtrar()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1,generos,apariciones,invalido,name
5393,8481,"""Bungaku Shoujo"" Memoire",7.34,"Drama, Romance, School",Unknown,“文学少女”メモワール,OVA,3,"Jun 25, 2010 to Dec 24, 2010",Unknown,...,1659.0,709.0,190.0,112.0,47.0,34.0,"[drama, romance, school]",1,False,"""bungaku shoujo"" memoire"
4584,6408,"""Bungaku Shoujo"" Movie",7.41,"Mystery, Drama, Romance, School",Unknown,劇場版“文学少女”,Movie,1,"May 1, 2010",Unknown,...,3021.0,1526.0,688.0,320.0,173.0,114.0,"[mystery, drama, romance, school]",1,False,"""bungaku shoujo"" movie"
4448,6076,"""Eiji""",5.76,"Comedy, Drama, Sports",Unknown,「エイジ」,OVA,1,"Aug 25, 1990",Unknown,...,193.0,176.0,76.0,44.0,15.0,22.0,"[comedy, drama, sports]",1,False,"""eiji"""
11863,33363,"""Eiyuu"" Kaitai",6.01,"Comedy, Drama",Eiyu Kaitai,「英雄」解体,OVA,1,"Nov 23, 2016",Unknown,...,437.0,263.0,109.0,54.0,23.0,22.0,"[comedy, drama]",1,False,"""eiyuu"" kaitai"
16224,40554,"""Uchuu Senkan Yamato"" to Iu Jidai: Seireki 220...",Unknown,"Action, Military, Sci-Fi, Space, Drama",Unknown,『宇宙戦艦ヤマト』という時代 西暦2202年の選択,Movie,1,Unknown,Unknown,...,2.0,2.0,Unknown,Unknown,Unknown,Unknown,"[action, military, sci-fi, space, drama]",1,False,"""uchuu senkan yamato"" to iu jidai: seireki 220..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12823,35099,Zukkoke Sannin-gumi no Jitensha Kyoushitsu,Unknown,"Drama, Kids",Unknown,ッコケ三人組の自転車教室,OVA,1,2016,Unknown,...,2.0,3.0,1.0,1.0,1.0,2.0,"[drama, kids]",1,False,zukkoke sannin-gumi no jitensha kyoushitsu
12869,35149,Zukkoke Sannin-gumi no Koutsuu Anzen,Unknown,"Drama, Kids",Unknown,ズッコケ三人組のこうつうあんぜん,OVA,1,Unknown,Unknown,...,1.0,1.0,2.0,2.0,Unknown,3.0,"[drama, kids]",1,False,zukkoke sannin-gumi no koutsuu anzen
14119,37207,Zuori Qing Kong,6.76,"Drama, Romance, School, Slice of Life",Crystal Sky of Yesterday,昨日青空,Movie,1,"Oct 26, 2018",Unknown,...,230.0,123.0,58.0,12.0,13.0,10.0,"[drama, romance, school, slice of life]",1,False,zuori qing kong
14816,38285,Zuori Qing Kong Pilot,5.94,"Drama, Romance, School, Slice of Life",Crystal Sky of Yesterday,昨日青空（动画样片）,ONA,1,"Jan 21, 2014",Unknown,...,52.0,37.0,9.0,7.0,9.0,6.0,"[drama, romance, school, slice of life]",1,False,zuori qing kong pilot


In [52]:
%timeit df[df.apply(lambda x:tiene_generos(x,["drama","game"]),axis=1)]

899 ms ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%timeit filtrar()

60.1 ms ± 3.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Si comparamos los tiempos de ejecucion, por mas que es complicado y se hacen varias cosas entre medio, la version que no recurre a apply es mucho mas eficiente en terminos de tiempo.

Ahora, siendo que el DataFrame con las series y generos tiene el mismo largo que el dataset original, podria concatenar las columnas del primero en el segundo.

In [56]:
df_final = pd.concat([df,serie_genero.reset_index()],axis=1)

Y con esto, ahora puedo hacer el mismo filtrado que antes, pero de forma mucho mas eficiente

In [58]:
def tiene_generos_v3(generos):
    result = df_final[generos[0]]

    for x in generos[1:]:
        result = result | df_final[x] 
    
    return result

df_final[ tiene_generos_v3(["drama","game"]) ]

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,slice of life,space,sports,super power,supernatural,thriller,unknown,vampire,yaoi,yuri
3,7.0,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,False,False,False,False,False,False,False,False,False,False
4,8.0,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,False,False,False,False,False,False,False,False,False,False
6,16.0,Hachimitsu to Clover,8.06,"Comedy, Drama, Josei, Romance, Slice of Life",Honey and Clover,ハチミツとクローバー,TV,24,"Apr 15, 2005 to Sep 27, 2005",Spring 2005,...,False,False,True,False,False,False,False,False,False,False
8,18.0,Initial D Fourth Stage,8.15,"Action, Cars, Sports, Drama, Seinen",Unknown,頭文字〈イニシャル〉D FOURTH STAGE,TV,24,"Apr 17, 2004 to Feb 18, 2006",Spring 2004,...,False,False,False,False,False,False,False,False,False,False
15,25.0,Sunabouzu,7.42,"Action, Adventure, Comedy, Ecchi, Sci-Fi, Shounen",Desert Punk,砂ぼうず,TV,24,"Oct 6, 2004 to Mar 30, 2005",Fall 2004,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17540,48413.0,Hataraku Maou-sama! 2nd Season,Unknown,"Comedy, Demons, Supernatural, Romance, Fantasy",Unknown,はたらく魔王さま！,TV,Unknown,Unknown,Unknown,...,False,False,False,False,False,False,False,False,False,False
17541,48414.0,Sabikui Bisco,Unknown,"Action, Adventure, Fantasy",Unknown,錆喰いビスコ,TV,Unknown,Unknown,Unknown,...,False,False,False,False,False,False,False,False,False,False
17546,48425.0,Akuma-kun (Shin Anime),Unknown,"Horror, Demons, Supernatural, Thriller",Unknown,悪魔くん,Unknown,Unknown,Unknown,Unknown,...,True,False,False,False,False,False,False,False,False,False
17547,48426.0,Kitarou Tanjou: Gegege no Nazo,Unknown,"Comedy, Demons, Supernatural, Shounen",Unknown,鬼太郎誕生 ゲゲゲの謎,Movie,1,Unknown,Unknown,...,True,False,False,False,False,False,False,False,False,False


In [ ]:
%timeit df_final[ tiene_generos_v3(["drama","game"]) ]

Por mucho que no lo parezca, si esta operacion la vamos a hacer mucho, es a la larga mas eficiente calcular esos datos para poder operar directamente sobre el df. Por el contrario, si es algo que vamos a hacer una unica vez, por muy ineficiente que sea la operacion en otro formato, quiza no justifica el tiempo de codeo.

#### Bonus (pregunta del kahoot)

In [59]:
df = pd.DataFrame( [ [x,3 if x>0 else 100000,3+(10000 if x%2 else -10000)] for x in range(10000000)])

In [60]:
df.describe().T["std"].map(lambda x: int(x))

0    2886751
1         31
2      10000
Name: std, dtype: int64